In [1]:
#Import Libraries
import pandas as pd
import numpy as np
import os

In [3]:
#Import path and csv files
path= r'/Users/owner/Documents/05_2023_Instacart_Basket_Analysis'

In [4]:
df_prods = pd.read_csv(os.path.join(path, '02 Data', 'Original Data', 'products.csv'), index_col = False)

In [5]:
df_ords = pd.read_csv(os.path.join(path, '02 Data', 'Original Data', 'orders.csv'), index_col = False)

In [6]:
df_ords_prior = pd.read_csv(os.path.join(path, '02 Data', 'Original Data', 'order_products_prior.csv'), index_col = False)

In [8]:
#Check data consistency 

In [9]:
#Check for missing values
df_ords_prior.isnull().sum()

order_id             0
product_id           0
add_to_cart_order    0
reordered            0
dtype: int64

In [10]:
# Check for full duplicates
df_dups = df_ords_prior[df_ords_prior.duplicated()]

In [11]:
df_dups

,order_id,product_id,add_to_cart_order,reordered


In [12]:
# Check for mixed data types in df_ords_prior
for col in df_ords_prior.columns.tolist():
  weird = (df_ords_prior[[col]].applymap(type) != df_ords_prior[[col]].iloc[0].apply(type)).any(axis = 1)
  if len (df_ords_prior[weird]) > 0:
    print (col)

In [13]:
#Descriptive Statistics for df_ords_priod
df_ords_prior.describe()

,order_id,product_id,add_to_cart_order,reordered
count,3.243449e+07,3.243449e+07,3.243449e+07,3.243449e+07
mean,1.710749e+06,2.557634e+04,8.351076e+00,5.896975e-01
std,9.873007e+05,1.409669e+04,7.126671e+00,4.918886e-01
min,2.000000e+00,1.000000e+00,1.000000e+00,0.000000e+00
25%,8.559430e+05,1.353000e+04,3.000000e+00,0.000000e+00
50%,1.711048e+06,2.525600e+04,6.000000e+00,1.000000e+00
75%,2.565514e+06,3.793500e+04,1.100000e+01,1.000000e+00
max,3.421083e+06,4.968800e+04,1.450000e+02,1.000000e+00


In [14]:
df_ords_prior.shape

(32434489, 4)

In [15]:
#Check output
df_ords_prior.head()

,order_id,product_id,add_to_cart_order,reordered
0,2,33120,1,1
1,2,28985,2,1
2,2,9327,3,0
3,2,45918,4,1
4,2,30035,5,0


In [16]:
#Check df_ords shape and first five rows
df_ords.shape

(3421083, 7)

In [17]:
df_ords.head()

,order_id,user_id,eval_set,order_number,order_dow,order_hour_of_day,days_since_prior_order
0,2539329,1,prior,1,2,8,NaN
1,2398795,1,prior,2,3,7,15.0
2,473747,1,prior,3,3,12,21.0
3,2254736,1,prior,4,4,7,29.0
4,431534,1,prior,5,4,15,28.0


df_ords_prior does not seem to be missing value or hold duplicates or mixed values, so it's ready to be merged with df_ords

In [18]:
#Merge data
df_merged_large = df_ords.merge(df_ords_prior, on = 'order_id', indicator = True)

In [19]:
#Check output of merged dataframes
df_merged_large.head()

,order_id,user_id,eval_set,order_number,order_dow,order_hour_of_day,days_since_prior_order,product_id,add_to_cart_order,reordered,_merge
0,2539329,1,prior,1,2,8,NaN,196,1,0,both
1,2539329,1,prior,1,2,8,NaN,14084,2,0,both
2,2539329,1,prior,1,2,8,NaN,12427,3,0,both
3,2539329,1,prior,1,2,8,NaN,26088,4,0,both
4,2539329,1,prior,1,2,8,NaN,26405,5,0,both


In [23]:
df_merged_large.shape

(32434489, 11)

In [20]:
#Check value counts 
df_merged_large['_merge'].value_counts()

both          32434489
left_only            0
right_only           0
Name: _merge, dtype: int64

In [21]:
#Check value size and rows
df_merged_large.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 32434489 entries, 0 to 32434488
Data columns (total 11 columns):
 #   Column                  Dtype   
---  ------                  -----   
 0   order_id                int64   
 1   user_id                 int64   
 2   eval_set                object  
 3   order_number            int64   
 4   order_dow               int64   
 5   order_hour_of_day       int64   
 6   days_since_prior_order  float64 
 7   product_id              int64   
 8   add_to_cart_order       int64   
 9   reordered               int64   
 10  _merge                  category
dtypes: category(1), float64(1), int64(8), object(1)
memory usage: 2.7+ GB


In [22]:
# Export data to pkl

df_merged_large.to_pickle(os.path.join(path, '02 Data','Prepared Data', 'orders_products_combined.pkl'))